In [ ]:
from vacation.model import VCNN
from vacation.data import GalaxyDataset
import torch
from torchinfo import summary

In [ ]:
train_ds = GalaxyDataset(
    path="../../.data/Galaxy10_DECals_train.h5",
    device="cuda",
    max_cache_size="7G",
    cache_loaded=False,
)
valid_ds = GalaxyDataset(
    path="../../.data/Galaxy10_DECals_train.h5",
    device="cuda",
    max_cache_size="2G",
    cache_loaded=False,
)

In [ ]:
model = VCNN(
    img_size=256,
    num_labels=10,
    train_batch_size=256,
    valid_batch_size=256,
    out_channels=[4, 8, 10],
    dropout_rates=[0, 0, 0],
    lin_out_features=[300, 100],
    optimizer=torch.optim.AdamW,
    activation_func=torch.nn.PReLU,
    metrics={},
    learning_rate=0.005,
    loss_func=torch.nn.CrossEntropyLoss,
)

In [ ]:
model.train_epochs(n_epochs=10, train_dataset=train_ds, valid_dataset=valid_ds)

In [ ]:
model._loss_metric

In [ ]:
summary(model, (3, 256, 256))